In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
from desc.monitor import RefLightCurves

/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [3]:
import os

In [4]:
import numpy as np
import pandas as pd

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

/usr/local/manual/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [6]:
import desc.monitor as monitor

In [7]:
monitor.__file__

'/Users/rbiswas/doc/projects/desc/Monitor/python/desc/monitor/__init__.pyc'

In [8]:
monitor.__file__.split('/python')[0]

'/Users/rbiswas/doc/projects/desc/Monitor'

In [9]:
data_dir = os.path.join(monitor.__file__.split('/python')[0], 'data')

In [10]:
opsimCsv = os.path.join(data_dir, 'SelectedKrakenVisits.csv')

In [11]:
opsimdf = pd.read_csv(opsimCsv, index_col='obsHistID')

In [12]:
df = opsimdf[['expMJD', 'filter', 'fiveSigmaDepth']]

In [13]:
df.index

Int64Index([    220,     230,     200,     276,     250,    1637,    2226,
               2942,    3665,    4571,
            ...
            2440831, 2440841, 2440811, 2440887, 2440861, 2442752, 2442772,
            2442742, 2442799, 2442775],
           dtype='int64', name=u'obsHistID', length=1227)

In [14]:
import pymssql
from lsst.utils import getPackageDir
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.daf.persistence import DbAuth

In [15]:
config = bcm.BaseCatalogConfig()

In [16]:
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))

In [17]:
username = DbAuth.username(config.host, config.port)    
password = DbAuth.password(config.host, config.port)
DBConnection = pymssql.connect(user=username,
                               password=password,
                               database=config.database,
                               port=config.port)
db = DBConnection.cursor()
reflc = RefLightCurves(idSequence=(6001163623700, 6000324908000),
                       tableName='TwinkSN',
                       dbConnection=DBConnection,
                       dbCursor=db)

In [18]:
sn = reflc.astro_object(idValue=6001163623700)

In [29]:
sn.SNstate

{'MWE(B-V)': 0.0082679092884063721,
 'ModelSource': 'salt2-extended',
 '_dec': -0.48595833295788454,
 '_ra': 0.92158758150141751,
 'c': -0.043500407073100003,
 'hostebv': 0.0,
 'hostr_v': 3.1000000000000001,
 'mwebv': 0.0,
 'mwr_v': 3.1000000000000001,
 't0': 61311.094089960003,
 'x0': 2.4638469756499999e-05,
 'x1': -1.98170765603,
 'z': 0.34010000000000001}

In [19]:
from lsst.sims.photUtils import BandpassDict

In [20]:
lsstBP = BandpassDict.loadBandpassesFromFiles()

In [21]:
reflc.lightCurve(idValue=6001163623700, observations=df, bandpassDict=lsstBP[0])

,band,flux,fluxerr,m5,time
obsHistID,,,,,
1163962,g,1.679591e-12,7.196063e-11,23.603784,61285.264120
1163982,i,1.380898e-12,1.049428e-10,23.193676,61285.273819
1163952,r,7.850403e-13,6.779478e-11,23.667112,61285.258587
1164028,y,1.534653e-12,3.665787e-10,21.836442,61285.295717
1164008,z,1.267230e-12,1.630022e-10,22.716097,61285.286018
1168321,u,1.115277e-10,7.266888e-11,23.575874,61298.266570
1169136,u,1.170866e-10,9.147401e-11,23.329497,61299.229359
1172807,g,1.093165e-09,5.447195e-11,23.937104,61305.284817
1172827,i,1.401860e-09,7.780287e-11,23.548731,61305.294516
